In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json


# **Examen Final – Recuperación de la Información**

## **Nombre:**  Nelson Casa

### **Dataset:** arXiv (Cornell University – Kaggle)

**Descripción del entorno de trabajo**

Este notebook se ejecuta en Kaggle, utilizando directamente el dataset arXiv disponible en el entorno, evitando problemas de derechos de autor y descargas externas

## FASE 0 – Preparación del entorno

**Instalación de librerías necesarias**

Instalación de librerías necesarias para procesamiento de texto, embeddings y búsqueda vectorial.

In [2]:
pip install sentence-transformers faiss-cpu nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 86.1 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


**Importación de las librerías que serán utilizadas a lo largo del sistema de recuperación de información.**

In [3]:
import json
import numpy as np
import pandas as pd
import nltk
import faiss

from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

2026-01-28 17:29:19.254209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769621359.685528      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769621359.816543      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769621360.889814      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769621360.889852      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769621360.889854      55 computation_placer.cc:177] computation placer alr

Se carga el dataset arXiv directamente desde el directorio de Kaggle donde se monta automáticamente el dataset.

Para efectos computacionales, se trabaja con una muestra representativa del dataset
antes de aplicar el preprocesamiento de texto.

In [4]:
# Ruta al dataset en Kaggle
file_path = '/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json'

def load_arxiv_subset(path, limit=20000):
    docs = []
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if i >= limit:
                break
            # Cargamos cada línea como un objeto JSON
            item = json.loads(line)
            docs.append({
                'id': item['id'],
                'title': item['title'],
                'abstract': item['abstract']
            })
    return pd.DataFrame(docs)

# Ejecución de la carga
df = load_arxiv_subset(file_path)
print(f"Dataset cargado con {len(df)} registros.")
df.head()

Dataset cargado con 20000 registros.


,id,title,abstract
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-..."
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...


Exploración inicial del dataset

Se inspeccionan los campos disponibles en el dataset para seleccionar la información relevante.

## FASE 1 – Preprocesamiento de Datos

En esta fase se aplica el preprocesamiento de texto (normalización, tokenización, eliminación de stopwords y stemming) sobre el subconjunto del dataset cargado.

In [6]:
nltk.download("stopwords")

stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

def preprocess(text):
    tokens = text.lower().split()                # Normalización
    tokens = [t for t in tokens if t.isalpha()]  # Tokenización limpia
    tokens = [t for t in tokens if t not in stop_words]  # Stopwords
    tokens = [stemmer.stem(t) for t in tokens]   # Stemming
    return " ".join(tokens)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Aplicamos preprocesamiento SOLO al subconjunto

In [7]:
df["clean_text"] = df["abstract"].apply(preprocess)

df.head()

,id,title,abstract,clean_text
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,fulli differenti calcul perturb quantum chromo...
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...",describ new game use obtain character famili g...
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,evolut system describ dark matter field fluid ...
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,show determin stirl cycl number count unlabel ...
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,paper show comput use dyadic result consequ de...


## FASE 2 – Representación mediante Embeddings

En esta fase se generan embeddings semánticos para los documentos y se almacenan para su posterior búsqueda vectorial.

In [8]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Se utilizan embeddings densos para capturar similitud semántica entre documentos.

In [9]:
doc_embeddings = embedding_model.encode(
    df["clean_text"].tolist(),
    show_progress_bar=True
)

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

## FASE 3 – Recuperación Inicial (First-Stage Retrieval)

En esta fase se implementa un mecanismo de recuperación inicial utilizando búsqueda vectorial con FAISS.

En esta celda se crea un índice FAISS a partir de los embeddings de los documentos.  

In [10]:
import faiss
import numpy as np

dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(doc_embeddings))

En esta celda se define la función de recuperación inicial. La consulta se preprocesa, se convierte en un embedding y se utiliza el índice FAISS para recuperar los documentos más cercanos semánticamente, retornando un ranking preliminar de resultados.

In [12]:
def first_stage_retrieval(query, top_k=10):
    query_clean = preprocess(query)
    query_embedding = embedding_model.encode([query_clean])
    distances, indices = index.search(np.array(query_embedding), top_k)
    
    results = []
    for rank, idx in enumerate(indices[0]):
        results.append({
            "rank": rank + 1,
            "id": df.iloc[idx]["id"],
            "title": df.iloc[idx]["title"],
            "score": distances[0][rank]
        })
    return results

Ejecución de la recuperación inicial

In [22]:
# Ejemplo de recuperación inicial sin re-ranking
query_example = "machine learning for image classification"

initial_results = first_stage_retrieval(query_example, top_k=5)

for r in initial_results:
    print(f"{r['rank']} - {r['title']}")

1 - Learning from dependent observations
2 - The Mathematics
3 - Why prove things?
4 - Ensemble Learning for Free with Evolutionary Algorithms ?
5 - On the need to enhance physical insight via mathematical skills


## FASE 4 – Re-ranking de Resultados

En esta fase se reordenan los documentos recuperados inicialmente utilizando un modelo cross-encoder más preciso.

In [13]:
from sentence_transformers import CrossEncoder

reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

En esta celda se define la función de re-ranking. A partir de la consulta y los documentos recuperados inicialmente, se utiliza un modelo cross-encoder para recalcular la relevancia y reordenar los resultados, generando un ranking final más preciso.

In [14]:
def rerank(query, candidates):
    pairs = [
        (query, df[df["id"] == c["id"]]["abstract"].values[0])
        for c in candidates
    ]
    
    scores = reranker.predict(pairs)
    
    reranked = sorted(
        zip(candidates, scores),
        key=lambda x: x[1],
        reverse=True
    )
    
    return [
        {
            "rank": i + 1,
            "id": item[0]["id"],
            "title": item[0]["title"],
            "rerank_score": item[1]
        }
        for i, item in enumerate(reranked)
    ]

#### Ejecución del re-ranking de documentos

En esta celda se aplica el re-ranking sobre los documentos recuperados inicialmente, utilizando un modelo cross-encoder para obtener un ranking final más preciso.

In [23]:
# Ejecución del re-ranking para la misma consulta
reranked_results = rerank(query_example, initial_results)

for r in reranked_results:
    print(f"{r['rank']} - {r['title']}")

1 - Ensemble Learning for Free with Evolutionary Algorithms ?
2 - Learning from dependent observations
3 - On the need to enhance physical insight via mathematical skills
4 - The Mathematics
5 - Why prove things?


## FASE 5 – Simulación de Consultas

En esta fase se ejecutan múltiples consultas y se comparan los resultados antes y después del re-ranking.

In [19]:
queries = [
    "machine learning for image classification",
    "economic policy and financial markets",
    "neural networks optimization"
]

for q in queries:
    print(f"-------------------------------------")
    print(f"\nConsulta: {q}")
    
    initial_results = first_stage_retrieval(q, top_k=5)
    reranked_results = rerank(q, initial_results)
    
    print("\nResultados iniciales:")
    for r in initial_results:
        print(r["rank"], "-", r["title"])
    
    print("\nResultados después del re-ranking:")
    for r in reranked_results:
        print(r["rank"], "-", r["title"])

-------------------------------------

Consulta: machine learning for image classification

Resultados iniciales:
1 - Learning from dependent observations
2 - The Mathematics
3 - Why prove things?
4 - Ensemble Learning for Free with Evolutionary Algorithms ?
5 - On the need to enhance physical insight via mathematical skills

Resultados después del re-ranking:
1 - Ensemble Learning for Free with Evolutionary Algorithms ?
2 - Learning from dependent observations
3 - On the need to enhance physical insight via mathematical skills
4 - The Mathematics
5 - Why prove things?
-------------------------------------

Consulta: economic policy and financial markets

Resultados iniciales:
1 - Models of Financial Markets with Extensive Participation Incentives
2 - Stability of utility-maximization in incomplete markets
3 - Entropy Oriented Trading: A Trading Strategy Based on the Second Law of
  Thermodynamics
4 - The Local Fractal Properties of the Financial Time Series on the Polish
  Stock Excha

## FASE 6 – Evaluación del Sistema

En esta celda se definen las métricas de evaluación del sistema.  
Precision@k mide la proporción de documentos relevantes entre los primeros k resultados recuperados, mientras que Recall@k mide la proporción de documentos relevantes recuperados respecto al total de documentos relevantes.

In [26]:
def precision_at_k(retrieved_ids, relevant_ids, k):
    retrieved_k = retrieved_ids[:k]
    return len(set(retrieved_k) & set(relevant_ids)) / k

def recall_at_k(retrieved_ids, relevant_ids, k):
    retrieved_k = retrieved_ids[:k]
    return len(set(retrieved_k) & set(relevant_ids)) / len(relevant_ids)

En esta celda se calcula la métrica Precision@5 para la recuperación inicial y para el ranking final.  
Debido a la ausencia de juicios de relevancia reales en el dataset, se utilizan relevancias simuladas con el fin de analizar el impacto del re-ranking.

In [27]:
retrieved = [r["id"] for r in first_stage_retrieval(queries[0], top_k=10)]
reranked = [r["id"] for r in rerank(queries[0], first_stage_retrieval(queries[0], 10))]

relevant_docs = retrieved[:3]  # simulación

print("Precision@5 (Inicial):", precision_at_k(retrieved, relevant_docs, 5))
print("Precision@5 (Re-ranking):", precision_at_k(reranked, relevant_docs, 5))

Precision@5 (Inicial): 0.6
Precision@5 (Re-ranking): 0.2


## FASE 7 – Análisis de Resultados

- El preprocesamiento aplicado (normalización, eliminación de stopwords y stemming) permitió reducir ruido y estandarizar el texto de los abstracts, facilitando la generación de embeddings más consistentes.

- El uso de embeddings densos permitió capturar similitud semántica entre documentos y consultas, funcionando correctamente en dominios bien representados dentro del subconjunto del corpus.

- La recuperación inicial con FAISS mostró buen desempeño al recuperar documentos conceptualmente relacionados, especialmente en las consultas de economía y redes neuronales.

- La etapa de re-ranking mejoró el orden de relevancia en algunos casos, pero su impacto depende directamente de la calidad del conjunto inicial de documentos recuperados.

- En algunas consultas asociadas a categorías específicas, los resultados obtenidos pueden no reflejar completamente el dominio de la consulta. Esto se debe a que, para efectos computacionales, se trabajó con un subconjunto del dataset completo, lo cual puede haber omitido documentos relevantes de ciertos temas durante el proceso de muestreo.


- La evaluación mediante Precision@k se realizó con relevancias simuladas, ya que el dataset arXiv no incluye qrels. Por esta razón, el re-ranking no siempre mejora la métrica, lo cual es consistente con la ausencia de juicios de relevancia reales.

- En general, el sistema fue correctamente implementado y cumple con todas las fases del pipeline de Recuperación de Información. Las limitaciones observadas están asociadas al dataset y al muestreo, no a errores en el diseño del sistema.
